In [32]:
#import librerias necesarias

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import plotly.express as px
from scipy import stats
from scipy.stats import norm, skew # para estadisticas


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # limitar flotantes a 3 decimales


from subprocess import check_output


## Carga del dataset, con encoding latin1

In [33]:
df = pd.read_csv(r'data\test.csv',encoding='latin-1')
df.head()

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,15576,0.052,53y,0,0.009,2245.000,3,0,0,0,0.000
1,94308,0.022,57y,0,0.320,7672.000,26,0,1,0,2.000
2,53983,0.014,68y,0,0.139,4064.000,4,0,0,0,1.000
3,61359,0.019,74y,0,0.019,10500.000,27,0,0,0,0.000
4,71383,0.009,66y,0,9.000,NaN,5,0,0,0,0.000


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31442 entries, 0 to 31441
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Id                                    31442 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  31442 non-null  float64
 2   age                                   31442 non-null  object 
 3   NumberOfTime30-59DaysPastDueNotWorse  31442 non-null  int64  
 4   DebtRatio                             31442 non-null  float64
 5   MonthlyIncome                         25228 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       31442 non-null  int64  
 7   NumberOfTimes90DaysLate               31442 non-null  int64  
 8   NumberRealEstateLoansOrLines          31442 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  31442 non-null  int64  
 10  NumberOfDependents                    30635 non-null  float64
dtypes: float64(4), 

Revolving Utilization of Unsecured Lines (RUUL) -- Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits.

DebtRatio -- Monthly debt payments, alimony,living costs divided by monthy gross income

NumberOfOpenCreditLinesAndLoans

NumberOfTime30-59DaysPastDueNotWorse

MonthlyIncome

NumberOfTimes90DaysLate

NumberRealEstateLoansOrLines

NumberOfTime60-89DaysPastDueNotWorse

NumberOfDependents

age

# Data Visualization and Feature Engineering

In [35]:
df['age'] = df['age'].str.replace('y', '').astype(int)


In [36]:
df.columns = df.columns.str.lower()

In [37]:
def data_report(df):
    # Sacamos los NOMBRES
    cols = pd.DataFrame(df.columns.values, columns=["COL_N"])

    # Sacamos los TIPOS
    types = pd.DataFrame(df.dtypes.values, columns=["DATA_TYPE"])

    # Sacamos los MISSINGS
    percent_missing = round(df.isnull().sum() * 100 / len(df), 2)
    percent_missing_df = pd.DataFrame(percent_missing.values, columns=["MISSINGS (%)"])

    # Sacamos los VALORES UNICOS
    unicos = pd.DataFrame(df.nunique().values, columns=["UNIQUE_VALUES"])
    
    percent_cardin = round(unicos['UNIQUE_VALUES']*100/len(df), 2)
    percent_cardin_df = pd.DataFrame(percent_cardin.values, columns=["CARDIN (%)"])

    concatenado = pd.concat([cols, types, percent_missing_df, unicos, percent_cardin_df], axis=1, sort=False)
    concatenado.set_index('COL_N', drop=True, inplace=True)


    return concatenado.T
        
data_report(df)

COL_N,id,revolvingutilizationofunsecuredlines,age,numberoftime30-59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60-89dayspastduenotworse,numberofdependents
DATA_TYPE,int64,float64,int32,int64,float64,float64,int64,int64,int64,int64,float64
MISSINGS (%),0.000,0.000,0.000,0.000,0.000,19.760,0.000,0.000,0.000,0.000,2.570
UNIQUE_VALUES,31442,26892,81,13,27036,6769,47,17,17,10,11
CARDIN (%),100.000,85.530,0.260,0.040,85.990,21.530,0.150,0.050,0.050,0.030,0.030


In [38]:
# Luego de analizar la columna 'monthlyincome', decido quitarla por lo siguiente: 
# 1. Tiene muchos NaNs
# 2. HAy indicadores que la consideran, como el debtratio. 
# 3. No especifica si es ingreso bruto o neto
# Tambien quitamos la columna "id"
# df = df.drop(columns = 'monthlyincome')
df = df.drop(columns = 'id')
# df = df.drop(columns= 'numberofdependents')

In [39]:
df.describe()

,revolvingutilizationofunsecuredlines,age,numberoftime30-59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60-89dayspastduenotworse,numberofdependents
count,31442.000,31442.000,31442.000,31442.000,25228.000,31442.000,31442.000,31442.000,31442.000,30635.000
mean,4.942,52.324,0.429,343.443,6766.742,8.480,0.268,1.019,0.244,0.757
std,167.667,14.731,4.252,1166.519,23685.365,5.095,4.228,1.138,4.213,1.113
min,0.000,21.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.030,41.000,0.000,0.179,3380.000,5.000,0.000,0.000,0.000,0.000
50%,0.152,52.000,0.000,0.369,5391.000,8.000,0.000,1.000,0.000,0.000
75%,0.560,63.000,0.000,0.881,8166.000,11.000,0.000,2.000,0.000,1.000
max,13930.000,109.000,98.000,61907.000,3008750.000,58.000,98.000,54.000,98.000,10.000


In [40]:
# df['numberofdependents'].describe()

In [41]:
# df['numberofdependents'].value_counts()

In [42]:
# df['numberofdependents'].mean()

In [43]:
mode_dependents = df['numberofdependents'].mode()[0]
df['numberofdependents'].fillna(mode_dependents, inplace=True)

In [44]:
df['monthlyincome'].fillna(df['monthlyincome'].quantile(0.75), inplace=True)

In [49]:
data_report(df)

COL_N,revolvingutilizationofunsecuredlines,age,numberoftime30-59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60-89dayspastduenotworse,numberofdependents
DATA_TYPE,float64,int32,int64,float64,float64,int64,int64,int64,int64,float64
MISSINGS (%),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
UNIQUE_VALUES,26892,81,13,27036,6769,47,17,17,10,11
CARDIN (%),85.530,0.260,0.040,85.990,21.530,0.150,0.050,0.050,0.030,0.030


**Chequeamos outliers**

In [46]:
# sns.boxplot(x=df["debtratio"])

In [47]:
# fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(20, 8))
# sns.boxplot(df, x='age', ax=axs[0,1])
# sns.boxplot(df, x='numberofopencreditlinesandloans', ax=axs[1,0])
# sns.boxplot(df, x='revolvingutilizationofunsecuredlines', ax=axs[0,0])
# sns.boxplot(df, x='numberofdependents', ax=axs[1,1])
# sns.boxplot(df, x='numberoftimes90dayslate', ax=axs[2,0])
# sns.boxplot(df, x='numberrealestateloansorlines', ax=axs[2,1])
# axs[0, 0].set_xticklabels(axs[0, 0].get_xticklabels(), rotation=90)
# axs[0, 1].set_xticklabels(axs[0, 1].get_xticklabels(), rotation=90)
# axs[1, 0].set_xticklabels(axs[1, 0].get_xticklabels(), rotation=90)
# axs[1, 1].set_xticklabels(axs[1, 1].get_xticklabels(), rotation=90)
# axs[1, 1].set_xticklabels(axs[2, 0].get_xticklabels(), rotation=90)
# axs[1, 1].set_xticklabels(axs[2, 1].get_xticklabels(), rotation=90);

In [50]:
df = df.drop(columns = ['numberoftime30-59dayspastduenotworse', 'numberoftime60-89dayspastduenotworse'])

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31442 entries, 0 to 31441
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   revolvingutilizationofunsecuredlines  31442 non-null  float64
 1   age                                   31442 non-null  int32  
 2   debtratio                             31442 non-null  float64
 3   monthlyincome                         31442 non-null  float64
 4   numberofopencreditlinesandloans       31442 non-null  int64  
 5   numberoftimes90dayslate               31442 non-null  int64  
 6   numberrealestateloansorlines          31442 non-null  int64  
 7   numberofdependents                    31442 non-null  float64
dtypes: float64(4), int32(1), int64(3)
memory usage: 1.8 MB


In [ ]:
df.to_csv('Test_processed.csv', index=False)

## Correlaciones

In [ ]:
# corr = np.abs(df.corr()['target']).sort_values(ascending=True)
# print(corr)

In [ ]:
# Variables numericas vs target categórico
# sns.pairplot(df,
#             kind='scatter',
#             hue='target');

In [ ]:
# plt.figure(figsize=(10,10))
# sns.heatmap(df.corr(),
#             vmin=-1,
#             vmax=1,
#             center=0,
#             cmap=sns.diverging_palette(145, 280, s=85, l=25, n=10),
#             square=True,
#             annot=True,
#             linewidths=.5);

In [ ]:
# df = df.drop(columns = ['numberoftimes90dayslate', 'numberoftime60-89dayspastduenotworse'])